In [2]:
import numpy as np
import pandas as pd
from textacy import text_stats, make_spacy_doc
from datasets import Dataset, load_dataset
import matplotlib.pyplot as plt
import itertools

In [22]:
kept_prompts = pd.read_csv("UF10k_mixedbread_topics.csv") #import the prompts we decided to keep after topic modelling

In [99]:
data_human = pd.DataFrame(load_dataset("argilla/ultrafeedback-binarized-preferences-cleaned")["train"]).sample(n=15000, random_state=42)

In [100]:
data_human_short = data_human.sample(n=4670).reset_index(drop=True) # Get sample random sample of the same size as our metric data
data_human_long = data_human.sample(n=9340).reset_index(drop=True)

## Human Data for fine tuning:

In [31]:
# # Helper stuff:
# data_to_drop = [9849, 1941, 4635, 7232, 7789] #first 3 diff langs, last 2 had NAs in them

In [101]:
data_human_short['chosen'] = data_human_short['chosen'].apply(lambda x: x[1]['content'])
data_human_short['rejected'] = data_human_short['rejected'].apply(lambda x: x[1]['content'])
data_human_short = data_human_short.loc[:, ['prompt', 'chosen', 'chosen-rating', 'rejected', 'rejected-rating']]
# data_human = data_human.drop(data_to_drop).reset_index(drop=True)
data_human_short['differences'] = data_human_short.loc[:, 'chosen-rating'] - data_human_short.loc[:, 'rejected-rating']

data_human_short.to_csv('human_short.csv')

In [102]:
data_human_long['chosen'] = data_human_long['chosen'].apply(lambda x: x[1]['content'])
data_human_long['rejected'] = data_human_long['rejected'].apply(lambda x: x[1]['content'])
data_human_long = data_human_long.loc[:, ['prompt', 'chosen', 'chosen-rating', 'rejected', 'rejected-rating']]
# data_human = data_human.drop(data_to_drop).reset_index(drop=True)
data_human_long['differences'] = data_human_long.loc[:, 'chosen-rating'] - data_human_long.loc[:, 'rejected-rating']

data_human_long.to_csv('human_double.csv')

In [103]:
data_human['chosen'] = data_human['chosen'].apply(lambda x: x[1]['content'])
data_human['rejected'] = data_human['rejected'].apply(lambda x: x[1]['content'])
data_human = data_human.loc[:, ['prompt', 'chosen', 'chosen-rating', 'rejected', 'rejected-rating']]
# data_human = data_human.drop(data_to_drop).reset_index(drop=True)
data_human['differences'] = data_human.loc[:, 'chosen-rating'] - data_human.loc[:, 'rejected-rating']

data_human_diffsort = data_human.sort_values(by='differences', ascending=False)[:4670].reset_index(drop=True)
data_human_maxsort = data_human.sort_values(by='chosen-rating', ascending=False)[:4670].reset_index(drop=True)

data_human_diffsort.to_csv('human_diffsort.csv')
data_human_diffsort.to_csv('human_maxsort.csv')

# Metric Data for fine tuning:

## Linear Metric

In [3]:
data_linear = pd.read_csv('linear_metric_df.csv')
data_linear = data_linear.loc[:, ['prompt', 'chosen', 'rejected', 'accepted_linear_metric', 'rejected_linear_metric', 'diff_linear_metric']]
# np.where(data_linear.isna())
# data_linear.sort_values(by = 'accepted_linear_metric', ascending=False) # Judging by this, we have some foregin language issues that is distorting the score so we will
# # take these rows out.
# data_linear = data_linear.drop([9849, 1941, 4635, 7234, 7791]).reset_index(drop=True) #this goes into data_to_drop because of above

### Now, we need to swap around the parts where our score alledges the rejected is better than the chosen

In [4]:
def bad_idx_finder(DataFrame):
    return np.where(DataFrame.iloc[:, 3] < DataFrame.iloc[:, 4])[0]

In [5]:
def data_switcher(badidxs: list, DataFrame): 
    storage = {'metric_accept': [], 'metric_accept_score': [], 'metric_reject': [], 'metric_reject_score': []}

    #populate the storage dict
    for idx in badidxs:
        storage['metric_accept'].append(DataFrame.iloc[idx, 2]) 
        storage['metric_accept_score'].append(DataFrame.iloc[idx, 4])

        storage['metric_reject'].append(DataFrame.iloc[idx, 1])
        storage['metric_reject_score'].append(DataFrame.iloc[idx, 3])

    mask = DataFrame.index.isin(badidxs)

    DataFrame.loc[mask, DataFrame.columns[1]] = storage['metric_accept']
    DataFrame.loc[mask, DataFrame.columns[3]] = storage['metric_accept_score']

    DataFrame.loc[mask, DataFrame.columns[2]] = storage['metric_reject']
    DataFrame.loc[mask, DataFrame.columns[4]] = storage['metric_reject_score']

    DataFrame['absolute_difference'] = abs(DataFrame.iloc[:, -1])

    return DataFrame


In [447]:
# storage = {'linear_accept': [], 'linear_accept_score': [], 'linear_reject': [], 'linear_reject_score': []} #initiate dict to store values of indexes to switch

# for i in bad_lin_idx: #loop through to populate the dict
#     storage['linear_accept'].append(data_linear.iloc[i]['rejected']) 
#     storage['linear_reject'].append(data_linear.iloc[i]['chosen'])
#     storage['linear_accept_score'].append(data_linear.iloc[i]['rejected_linear_metric'])
#     storage['linear_reject_score'].append(data_linear.iloc[i]['accepted_linear_metric'])

# mask = data_linear.index.isin(bad_lin_idx) #make a index map for locing 

# # do the switching
# data_linear.loc[mask, 'chosen'] = storage['linear_accept']
# data_linear.loc[mask, 'rejected'] = storage['linear_reject']
# data_linear.loc[mask, 'accepted_linear_metric'] = storage['linear_accept_score']
# data_linear.loc[mask, 'rejected_linear_metric'] = storage['linear_reject_score']


# data_linear['absolute_difference'] = abs(data_linear['diff_linear_metric']) #absolute difference will reflect the updated values

In [6]:
linear_idxs = bad_idx_finder(data_linear)
data_linear = data_switcher(linear_idxs, data_linear)
data_linear_diffpreped = data_linear.sort_values(by='absolute_difference', ascending=False).reset_index(drop=True) # this dataset maximizes the difference in score between Chosen and Rejected
data_linear_maxpreped = data_linear.sort_values(by='accepted_linear_metric', ascending=False).reset_index(drop=True)# this dataset maximizes the raw Chosen score

In [9]:
# reversing the bunka score order as control

data_linear_diffpreped_reversed = pd.DataFrame({})
data_linear_diffpreped_reversed['prompt'] = data_linear_diffpreped['prompt']
data_linear_diffpreped_reversed['chosen'] = data_linear_diffpreped['rejected']
data_linear_diffpreped_reversed['rejected'] = data_linear_diffpreped['chosen']

In [12]:
data_linear_diffpreped.to_csv('diff_prepped_linear_metric.csv')
data_linear_maxpreped.to_csv('max_preped_linear_metric.csv')
data_linear_diffpreped_reversed.to_csv('diff_prepped_linear_metric_REVERSED.csv')

## PCA Metric

### Non-Weighted

In [335]:
data_pca = pd.read_csv('pca_metric_df.csv')
data_pca = data_pca.loc[:, ['prompt', 'chosen', 'rejected', 'accepted_pca1', 'rejected_pca1', 'diff_pca1']]
data_pca = data_pca.drop(data_to_drop).reset_index(drop=True)

In [357]:
pca_idxs = bad_idx_finder(data_pca)
data_pca = data_switcher(pca_idxs, data_pca)
data_pca_diff_prepped = data_pca.sort_values(by='absolute_difference', ascending=False)
data_pca_max_prepped = data_pca.sort_values(by='accepted_pca1', ascending=False)


### Weighted

In [8]:
data_weighted_pca = pd.read_csv('weighted_pca.csv')
data_weighted_pca = data_weighted_pca.loc[:, ['prompt', 'chosen', 'rejected', 'accepted_wpca', 'rejected_wpca', 'diff_wpca']]
data_weighted_pca = data_weighted_pca.drop(data_to_drop).reset_index(drop=True) #proabbly dont need the data to drop anymore because its taken care of on the backend

In [12]:
wpca_idxs = bad_idx_finder(data_weighted_pca)
data_weighted_pca = data_switcher(wpca_idxs, data_weighted_pca)
data_wpca_diff_prepped = data_weighted_pca.sort_values(by='absolute_difference', ascending=False)
data_wpca_max_prepped = data_weighted_pca.sort_values(by='accepted_wpca', ascending=False)